# 符合Python风格的对象

自定义类型的行为可以像内置类型那样自然，得益于Python的数据模型。

实现这样自然的行为，靠的不是继承，而是鸭子类型(duck typing)：我们只需按照预订行为实现对象所需方法即可。

## 对象表示形式

repr(): 以便于开发者理解的方式返回对象的字符串表示形式；对应`__repr__`

str(): 以便于用户理解的方式返回对象的字符串表示形式；对应`__str__`

In [ ]:
from array import array
import math

class Vector2d:
    __slots__ = ('__x', '__y')
    
    typecode = 'd'
    
    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)
    
    @property
    def x(self):
        return self.__x
    
    @property
    def y(self):
        return self.__y
    
    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __hash__(self):
        return hash(self.x) ^ hash(self.y)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)
    
    def angle(self):
        return math.atan2(self.y, self.x)
    
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)

v = Vector2d(3, 4)
print(repr(v))
print(v)
print(*v)
print(bool(v))
print(abs(v))
print(v == (3, 4))
print(format(v, '.3ep'))
print(hash(v))
set([v])
print(v.__slots__)
print(dir(v))

## classmethod与staticmethod

classmethod: 定义操作类，而不是操作实例的方法。

类方法改变了调用方法的方式，第一个参数是类本身，而不是实例。classmethod最常见用途是定义备选构造方法，第一个参数约定名称为cls。

staticmethod: 定义在类中的普通函数。

静态方法也改变了调用方法的方式，但第一个参数不是特殊的值。它就是一个定义在类中，而不是模块中的普通函数。

In [ ]:
class Demo:
    @classmethod
    def klassmeth(*args):
        return args
    
    @staticmethod
    def staticmeth(*args):
        return args
print(Demo.klassmeth())
print(Demo.klassmeth(1))
print(Demo.staticmeth())
print(Demo.staticmeth(2))

## 格式化显示

内置的format()函数和str.format()方法把各个类型的格式化方式委托给相应的`.__format__(format_spec)`方法。format_spec是格式说明符：

- format(myobj, format_spec)的第二个参数
- str.format()方法里的格式化字符串，{}里代换字段中：后面的部分。

格式说明符使用的表示法是格式规范微语言


## Python的私有属性和“受保护的”属性

Python不像Java那样使用private修饰符创建私有属性，但是有个简单机制，能够避免子类意外覆盖“私有”属性。

如果以`__mood`（双前导下划线，尾部没有或只有一个下划线）形式命名实例属性，Python会将属性名存入`__dict__`中，并在前面加上一个下划线和类名。

这叫做名称改写，只是一个保护措施，并不能真的保证安全。

也有人约定使用一个前导下划线定义受保护的属性，Python不会对单下划线的属性名做特殊处理。

## 使用`__slots__`类属性节省空间

Python默认在各个实例中名为`__dict__`的字典里存储实例属性。因为字典使用底层的散列表提升访问速度，故消耗大量内存。

如果处理大量的属性不多的实例，通过`__slots__`类属性，能节省大量内存。它会让解释器在元组中存储实例属性，而不用字典。

继承自超类的`__slots__`属性没有效果，Python只会使用各个类中定义的`__slots__`属性。

在类中定义`__slots__`属性的目的是告诉解释器：这个类中的所有实例属性都在这儿了！因而Python会在各个实例中使用类似元组的结构存储实例变量。

若要让实例可以作为weakref的目标，需要将`__weakref__`加入到`__slots__`中。

## 覆盖类属性

Python有个独特特性：类属性可用于为实例属性提供默认值。

当为不存在的实例属性赋值时，会新建实例属性，所以类属性可以被实例定制化。

若要改变类属性的默认值，常见做法是用拓展子类改变默认类属性的值。